# Инсталляция библиотек

In [83]:
pip install datasets evaluate

# Очистка данных

In [ ]:
# import pandas as pd
# import glob
# import chardet
# import os

# # Ищем все файлы сезонов
# season_files = sorted(glob.glob("/content/season*.csv"))

# df_list = []

# for file in season_files:
#     # Определяем кодировку перед чтением
#     with open(file, "rb") as f:
#         result = chardet.detect(f.read(100000))  # Анализ первых 100кб
#         encoding = result["encoding"]

#     print(f"Читаем файл {file} с кодировкой {encoding}")

#     try:
#         # Читаем CSV-файл с определенной кодировкой
#         df = pd.read_csv(file, encoding=encoding, on_bad_lines="skip")
#         df["season"] = os.path.basename(file).split(".")[0]  # Добавляем сезон
#         df_list.append(df)
#     except Exception as e:
#         print(f"Ошибка при чтении {file}: {e}")

# # Объединяем все файлы в один DataFrame
# if df_list:
#     full_dataset = pd.concat(df_list, ignore_index=True)
#     print(f"\nФинальный размер объединенного датасета: {full_dataset.shape}")

#     # Приведение имени персонажа к единому формату
#     full_dataset["name"] = full_dataset["name"].astype(str).str.strip().str.lower()

#     # Добавление колонок контекста
#     full_dataset["context_1"] = None
#     full_dataset["context_1_character"] = None
#     full_dataset["context_2"] = None
#     full_dataset["context_2_character"] = None
#     full_dataset["context_3"] = None
#     full_dataset["context_3_character"] = None

#     # Заполнение контекста только для строк, где name = 'house'
#     for i in range(3, len(full_dataset)):
#         if full_dataset.loc[i, "name"] == "house":
#             full_dataset.at[i, "context_1"] = full_dataset.loc[i - 1, "line"]
#             full_dataset.at[i, "context_1_character"] = full_dataset.loc[i - 1, "name"]

#             full_dataset.at[i, "context_2"] = full_dataset.loc[i - 2, "line"]
#             full_dataset.at[i, "context_2_character"] = full_dataset.loc[i - 2, "name"]

#             full_dataset.at[i, "context_3"] = full_dataset.loc[i - 3, "line"]
#             full_dataset.at[i, "context_3_character"] = full_dataset.loc[i - 3, "name"]

#     # Сохранение полного датасета с контекстом
#     full_output_file = "/content/house_full_dataset_with_context.csv"
#     full_dataset.to_csv(full_output_file, index=False, encoding="utf-8")
#     print(f"\nОбновленный датасет сохранен: {full_output_file}")

#     # Фильтрация реплик Доктора Хауса с контекстом
#     house_lines = full_dataset[full_dataset["name"] == "house"]

#     # Сохранение реплик Хауса
#     house_output_file = "/content/house_lines_with_context.csv"
#     house_lines.to_csv(house_output_file, index=False, encoding="utf-8")
#     print(f"Файл с репликами Хауса сохранен: {house_output_file}")
# else:
#     print("Ошибка: не удалось загрузить ни один файл!")


In [ ]:
import os
import shutil
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util, models, losses, evaluation, InputExample
from transformers import (
    AutoConfig, AutoModel, AutoTokenizer,
    AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import evaluate
from google.colab import files

# Определение устройства для работы с моделью (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Пути к данным
house_path = "/content/data/house_lines_with_context.csv"
antagonists_path = "/content/data/all_series_cleaned.csv"

# Загрузка данных
house_df = pd.read_csv(house_path)
antagonists_df = pd.read_csv(antagonists_path)

# Вывод первых строк для проверки
print("Первые строки датасета Доктора Хауса:")
print(house_df.head())

print("Первые строки датасета антагонистов:")
print(antagonists_df.head())

Первые строки датасета Доктора Хауса:
    name                                               line   season  \
0  house   See that? They all assume I'm a Patient becau...  season1   
1  house           I don't want them to think I'm a doctor.  season1   
2  house                   People don't want a sick doctor.  season1   
3  house         The one who can't talk, I liked that part.  season1   
4  house   And your cousin doesn't like the diagnosis. I...  season1   

                                           context_1 context_1_character  \
0   29 year old female, first seizure one month a...              wilson   
1        So put on a white coat like the rest of us.              wilson   
2   You see where the administration might have a...              wilson   
3   Fair enough. I don't like healthy Patients. T...              wilson   
4                                   She's my cousin.              wilson   

                                           context_2 context_2_character

In [ ]:
import pandas as pd
import re

# --- ФУНКЦИИ ОЧИСТКИ ---
def clean_text(text):
    """Очистка текста от проблемных символов, лишних пробелов и мусора."""
    if not isinstance(text, str) or len(text) == 0:
        return ""

    text = text.replace("�", "")  # Удаление неизвестных символов
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Удаляем любые не-ASCII символы (если есть)

    return text

def additional_cleaning(text):
    """Дополнительная очистка с учетом разрешенных символов."""
    if not isinstance(text, str):
        return text

    text = re.sub(r"[^\w\s.,!?'$/%-]", "", text)  # Оставляем только буквы, цифры и базовые знаки препинания
    text = re.sub(r"\s+", " ", text).strip()  # Удаление лишних пробелов

    return text

def apply_text_cleaning(df, text_columns):
    """Очистка всех указанных текстовых колонок."""
    for col in text_columns:
        df[col] = df[col].astype(str).apply(clean_text).apply(additional_cleaning)
    return df

# --- ОЧИСТКА ДАННЫХ ---
# Определяем нужные колонки
text_columns_house = ["line", "context_1", "context_2", "context_3"]
text_columns_antagonists = ["line"]

# Применяем очистку к датасетам
house_df = apply_text_cleaning(house_df, text_columns_house)
antagonists_df = apply_text_cleaning(antagonists_df, text_columns_antagonists)

# Выводим итоговую статистику
print(f"\nКоличество реплик после очистки: {len(house_df)} (Доктор Хаус), {len(antagonists_df)} (Антагонисты).")



Количество реплик после очистки: 21826 (Доктор Хаус), 167345 (Антагонисты).


In [ ]:
def is_question(text):
    """Определяет, содержит ли реплика вопрос"""
    if not isinstance(text, str):
        return False

    # Проверка, заканчивается ли текст на вопросительный знак
    if text.strip().endswith("?"):
        return True

    # Поиск вопросительных слов в тексте
    question_words = {"what", "why", "how", "who", "when", "where", "which",
                      "что", "почему", "как", "кто", "когда", "где", "какой", "зачем"}

    words = set(text.lower().split())
    if question_words.intersection(words):
        return True

    # Проверка наличия вопросительного знака внутри реплики
    if "?" in text:
        return True

    # Проверка восклицательных предложений, содержащих вопросительные слова
    if text.endswith("!") and question_words.intersection(words):
        return True

    return False

def analyze_questions(df, context_columns):
    """Подсчитывает количество вопросов в указанных колонках"""
    question_stats = {col: df[col].apply(is_question).sum() for col in context_columns}
    return question_stats

# Определение колонок для анализа
context_columns = ["line", "context_1", "context_2", "context_3"]

# Анализ вопросов в контексте
question_stats = analyze_questions(house_df, context_columns)

# Вывод статистики по вопросам
print("\nКоличество вопросов в предыдущих репликах:")
for col, count in question_stats.items():
    total = len(house_df)
    print(f"{col}: {count} ({count / total * 100:.2f}%)")

# Выборка примеров реплик с вопросами
print("\nПримеры вопросов в предыдущих репликах:")
for col in context_columns:
    sample_questions = house_df[house_df[col].apply(is_question)][col].dropna().sample(5, random_state=42).tolist()
    print(f"\n{col}:")
    for q in sample_questions:
        print(f"- {q}")



Количество вопросов в предыдущих репликах:
line: 7983 (36.58%)
context_1: 7288 (33.39%)
context_2: 7595 (34.80%)
context_3: 7313 (33.51%)

Примеры вопросов в предыдущих репликах:

line:
- And if she whacks herself, what happens to your job?
- See? There are those eyes again. He moves toward the desk and Adams grabs Nick's folder and holds it away from House. How about a viral syndrome? Or MRSA? No, that wouldn't chew up the bone that fast. But reaction to meds could. holds out his hand for the chart May I?
- I am so ashamed. to Wilson She really that good looking?
- For you, or me?
- Turns back to the Patient. Any fEver?

context_1:
- What are what are you doing? What are you doing?
- Do you know why people beliEve in God?
- She was screaming she was being sucked into a black hole, so, we should look for a tumor in her anus?
- Where are we going?
- interjects loudly PRobably? You think there's a possibility that the alternative is true?

context_2:
- looking at his yo-yo I can't remem

In [ ]:
# Функция для вычисления статистики по длине текста
def text_length_stats(df, text_columns):
    stats = {}
    for col in text_columns:
        lengths = df[col].dropna().astype(str).apply(len)  # Длина строк
        stats[col] = {
            "min": lengths.min(),
            "max": lengths.max(),
            "mean": lengths.mean(),
            "median": lengths.median(),
            "std": lengths.std()
        }
    return pd.DataFrame(stats)

# Определяем колонки для анализа
text_columns = ["line", "context_1", "context_2", "context_3"]

# Вычисляем статистику
stats_df = text_length_stats(house_df, text_columns)

stats_df

,line,context_1,context_2,context_3
min,2.000000,0.000000,0.000000,0.000000
max,1293.000000,1293.000000,1079.000000,1293.000000
mean,84.425822,62.191286,77.118483,64.355906
median,61.000000,45.000000,56.000000,46.000000
std,80.980381,59.355878,74.420721,61.907518


In [ ]:
import re

def truncate_long_text(text, max_words=30):
    """Обрезает текст после первого найденного знака препинания (. ? !) или по количеству слов."""
    if not isinstance(text, str):
        return text

    words = text.split()

    # Если реплика короче max_words, оставляем как есть
    if len(words) <= max_words:
        return text

    # Ищем ближайший знак препинания (. ? !)
    match = re.search(r"([.!?])\s", text)

    if match:
        cutoff_index = match.end()  # Обрезаем после знака
        return text[:cutoff_index].strip()

    # Если знаков нет — оставляем первые max_words слов
    return " ".join(words[:max_words]).strip()

# Применяем к колонке 'line' (ответы Хауса)
house_df["line"] = house_df["line"].apply(lambda x: truncate_long_text(x, max_words=30))

print("✅ Длинные реплики Хауса обрезаны по знакам препинания или количеству слов (30 слов макс.).")


✅ Длинные реплики Хауса обрезаны по знакам препинания или количеству слов (30 слов макс.).


In [ ]:
# Функция для подсчёта слов в реплике
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Подсчитываем количество реплик, где больше 50 слов
long_replies = house_df[house_df["line"].apply(count_words) > 50]

# Количество таких реплик
long_replies_count = len(long_replies)

print(f"📊 Количество реплик, где >50 слов: {long_replies_count}\n")

# Выводим 10 самых длинных реплик
longest_replies = long_replies.sort_values(by="line", key=lambda x: x.str.len(), ascending=False).head(10)

longest_replies

📊 Количество реплик, где >50 слов: 0



,name,line,season,context_1,context_1_character,context_2,context_2_character,context_3,context_3_character


In [ ]:
# Функция для подсчёта слов в тексте
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Создаём колонку anchor: копируем context_1, если он больше 10 слов
house_df["anchor"] = house_df["context_1"].apply(lambda x: x if count_words(x) > 10 else "")

# Подсчитываем количество строк с длинными anchor (> 50 слов)
long_anchor_count = (house_df["anchor"].apply(count_words) > 50).sum()

# Вывод статистики
print(f"Количество строк, где 'anchor' не пустой: {house_df['anchor'].str.len().gt(0).sum()}")
print(f"Количество слишком длинных 'anchor' (>50 слов): {long_anchor_count}")

Количество строк, где 'anchor' не пустой: 8638
Количество слишком длинных 'anchor' (>50 слов): 252


In [ ]:
import re

def truncate_anchor(text, max_words=50):
    """Обрезает anchor, оставляя 50 слов с конца + до ближайшего знака препинания (.?!)."""
    if not isinstance(text, str):
        return text

    words = text.split()

    # Если текст короче max_words, оставляем как есть
    if len(words) <= max_words:
        return text

    # Оставляем последние max_words слов
    truncated_text = " ".join(words[-max_words:])

    # Ищем ближайший знак препинания (вперёд от обрезанного конца)
    match = re.search(r"[.!?]", truncated_text)

    if match:
        cutoff_index = match.end()  # Обрезаем после знака
        return truncated_text[:cutoff_index].strip()

    return truncated_text.strip()  # Если знаков нет, оставляем как есть

# Применяем обрезку только к слишком длинным anchor
house_df["anchor"] = house_df["anchor"].apply(lambda x: truncate_anchor(x) if count_words(x) > 50 else x)

# Проверяем, сколько длинных anchor осталось
long_anchor_count_after = (house_df["anchor"].apply(count_words) > 70).sum()

print(f"Количество слишком длинных 'anchor' (>70 слов) после обрезки: {long_anchor_count_after}")

Количество слишком длинных 'anchor' (>70 слов) после обрезки: 0


In [ ]:
# Фильтруем строки, где anchor пустой
empty_anchor_df = house_df[house_df["anchor"].str.len() == 0]

# Подсчитываем количество пустых anchor
empty_anchor_count = len(empty_anchor_df)

# Статистика по длине context_1 и context_2 в этих строках
context_stats = {
    "context_1": {
        "min": empty_anchor_df["context_1"].str.len().min(),
        "max": empty_anchor_df["context_1"].str.len().max(),
        "mean": empty_anchor_df["context_1"].str.len().mean(),
        "median": empty_anchor_df["context_1"].str.len().median(),
        "std": empty_anchor_df["context_1"].str.len().std(),
    },
    "context_2": {
        "min": empty_anchor_df["context_2"].str.len().min(),
        "max": empty_anchor_df["context_2"].str.len().max(),
        "mean": empty_anchor_df["context_2"].str.len().mean(),
        "median": empty_anchor_df["context_2"].str.len().median(),
        "std": empty_anchor_df["context_2"].str.len().std(),
    }
}

# Вывод статистики
print(f"Количество строк с пустым 'anchor': {empty_anchor_count}\n")
print("Статистика по длине контекста:")
for col, stats in context_stats.items():
    print(f"\n{col}:")
    for key, value in stats.items():
        print(f"  {key}: {value}")


Количество строк с пустым 'anchor': 13188

Статистика по длине контекста:

context_1:
  min: 0
  max: 93
  mean: 29.56756141947225
  median: 28.0
  std: 14.946127577574105

context_2:
  min: 2
  max: 1079
  mean: 76.40498938428875
  median: 54.0
  std: 75.7249535767355


In [ ]:
# Функция для подсчёта слов
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Фильтруем строки для каждого количества слов
for i in range(10, 4, -1):  # От 10 до 5 слов
    sample = house_df[house_df["context_1"].apply(count_words) == i].head(5)
    print(f"\nПримеры 'context_1' с {i} словами:")
    print(sample["context_1"].tolist())


Примеры 'context_1' с 10 словами:
['I was expecting you in my office 20 minutes ago.', "You're 6 years behind on your obligation to this clinic.", 'I made sure your first case was an interesting one.', 'heavy sigh Ya know, I think there just might be.', 'Yes, yes, her name is Rebecca. I call her Rachel.']

Примеры 'context_1' с 9 словами:
['Psitticosis can lead to nerve pRoblems and neurological complications.', "Whoa, oh, I can't just break into someone's House.", 'Not that I could tell from her underwear drawer.', 'I just want to die with a little dignity.', 'I worked very hard to get where I am.']

Примеры 'context_1' с 8 словами:
['You think we have nothing to talk about?', "You also can't make long distance phone calls.", 'I was thinking it also might be fibromyalgia.', 'I solved the case, my work is done.', 'You hired me to get into my pants?!']

Примеры 'context_1' с 7 словами:
["And she's not responding to radiation treatment.", "Isn't treating Patients why we became doctors?"

In [ ]:
# Фильтруем строки для каждого количества слов
for i in range(4, 0, -1):  # От 4 до 1 слов
    sample = house_df[house_df["context_1"].apply(count_words) == i].head(5)
    print(f"\nПримеры 'context_1' с {i} словами:")
    print(sample["context_1"].tolist())



Примеры 'context_1' с 4 словами:
['I sign your paychecks.', 'I thought Everybody lied?', "You couldn't have knocked?", "And if we're wrong?", "She's a kindergarten teacher!"]

Примеры 'context_1' с 3 словами:
["She's my cousin.", 'No family history.', "It's a lesion.", 'Patient is orange.', 'No, the fruit.']

Примеры 'context_1' с 2 словами:
['No, but!', 'Mad cow?', "Wernickie's encephalopathy?", 'To what?', "It's cold."]

Примеры 'context_1' с 1 словами:
['What?!', '20', 'Right.', 'Nothing.', 'Hey!listen!']


In [ ]:
# Обновляем логику заполнения anchor: теперь переносим context_1, если в нем 3+ слова
house_df["anchor"] = house_df["context_1"].apply(lambda x: x if count_words(x) >= 3 else "")

# Подсчитываем количество строк, где anchor не пустой
filled_anchor_count = house_df["anchor"].str.len().gt(0).sum()

# Подсчитываем количество строк, где anchor остался пустым
empty_anchor_df = house_df[house_df["anchor"].str.len() == 0]
empty_anchor_count = len(empty_anchor_df)

# Подсчитываем сколько строк с пустым anchor имеют 0, 1 или 2 слова в context_1
empty_anchor_stats = {
    words: (empty_anchor_df["context_1"].apply(count_words) == words).sum()
    for words in [0, 1, 2]
}

# Вывод статистики
print(f"Количество строк, где 'anchor' не пустой: {filled_anchor_count}")
print(f"Количество строк с пустым 'anchor': {empty_anchor_count}")
print("\nРаспределение среди пустых 'anchor' по количеству слов в 'context_1':")
for key, value in empty_anchor_stats.items():
    print(f"  {key} слов(а): {value}")

Количество строк, где 'anchor' не пустой: 19842
Количество строк с пустым 'anchor': 1984

Распределение среди пустых 'anchor' по количеству слов в 'context_1':
  0 слов(а): 3
  1 слов(а): 870
  2 слов(а): 1111


In [ ]:
# Удаляем строки, где context_1 пустой (0 слов)
house_df = house_df[house_df["context_1"].str.len() > 0].reset_index(drop=True)

# Фильтруем строки, где context_1 содержит 1 или 2 слова
short_context_df = house_df[house_df["context_1"].apply(count_words).isin([1, 2])]

# Группируем и считаем самые частые context_1
top_short_contexts = (
    short_context_df.groupby("context_1").size().reset_index(name="count").sort_values(by="count", ascending=False).head(10)
)

# Выводим примеры (контекст + ответ Хауса)
print("\n10 самых часто встречающихся коротких context_1 (1-2 слова) с ответами Хауса:\n")
for _, row in top_short_contexts.iterrows():
    context = row["context_1"]
    count = row["count"]
    replies = short_context_df[short_context_df["context_1"] == context]["line"].unique()[:3]  # Берём 3 уникальных ответа Хауса
    print(f"'{context}' (встречается {count} раз)")
    for reply in replies:
        print(f"  - {reply}")
    print()



10 самых часто встречающихся коротких context_1 (1-2 слова) с ответами Хауса:

'No.' (встречается 108 раз)
  - But you are damaged, aren't you?
  - Well, then, Dave!
  - Here on work?

'What?' (встречается 54 раз)
  - Don't move. Did I bore you in there?
  - Husband described her as being unusually irritable recently.
  - Nice respectful Asian kid does the laundry.

'Why?' (встречается 40 раз)
  - 'Cause we need all the information we can get. The healthy kid can be our control group.
  - I find your interest interesting.
  - Cause the husband's not sick.

'Yeah.' (встречается 36 раз)
  - How'd you talk him down?
  - Cheers. Dry swallows the pill and Funsten eats his ewwwww Limps back
  - Now, why would you drive 70 miles to get treatment for a condition that a 9 year old could diagnose? It's the free-flowing pus that's the tip-off.

'Yes.' (встречается 29 раз)
  - Too bad, you're right.
  - It's pRobably an allergic reaction.
  - After you got so freaked about the sick babies a while

In [ ]:
import re

# Функция для извлечения последнего предложения из текста
def get_last_sentence(text):
    if not isinstance(text, str) or len(text) == 0:
        return ""
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Разделяем текст на предложения
    return sentences[-1] if sentences else text  # Берём последнее предложение

# Обновляем anchor: если он пустой, добавляем последнее предложение из context_2 + context_1
house_df["anchor"] = house_df.apply(
    lambda row: row["anchor"] if row["anchor"] else f"{get_last_sentence(row['context_2'])} {row['context_1']}".strip(), axis=1
)

# Подсчитываем количество оставшихся пустых anchor
empty_anchor_count_after = house_df["anchor"].str.len().eq(0).sum()

# Выводим статистику
print(f"Количество оставшихся пустых 'anchor' после заполнения: {empty_anchor_count_after}")


Количество оставшихся пустых 'anchor' после заполнения: 0


In [ ]:
# Подсчёт слов в anchor
house_df["anchor_word_count"] = house_df["anchor"].apply(count_words)

# Подсчитываем количество коротких anchor (<5 слов)
short_anchor_count = (house_df["anchor_word_count"] < 5).sum()

# Выводим статистику по длине anchor
anchor_stats = {
    "min": house_df["anchor_word_count"].min(),
    "max": house_df["anchor_word_count"].max(),
    "mean": house_df["anchor_word_count"].mean(),
    "median": house_df["anchor_word_count"].median(),
    "std": house_df["anchor_word_count"].std(),
}

# Вывод результатов
print(f"Количество anchor с менее чем 5 словами: {short_anchor_count}\n")
print("Статистика по длине anchor:")
for key, value in anchor_stats.items():
    print(f"  {key}: {value}")

# Вывести 10 самых коротких anchor
shortest_anchors = house_df[house_df["anchor_word_count"] < 5][["anchor", "line"]].head(10)
print("\n10 самых коротких anchor:\n")
for _, row in shortest_anchors.iterrows():
    print(f"Anchor: {row['anchor']}")
    print(f"House: {row['line']}\n")


Количество anchor с менее чем 5 словами: 3646

Статистика по длине anchor:
  min: 2
  max: 238
  mean: 12.101865004811437
  median: 9.0
  std: 10.661105791930531

10 самых коротких anchor:

Anchor: She's my cousin.
House: And your cousin doesn't like the diagnosis. I wouldn't either. Brain tumor, she's gonna die, boring.

Anchor: No family history.
House: I thought your uncle died of cancer.

Anchor: It's a lesion.
House: And the big green thing in the middle of the bigger blue thing on a map is an island. I was hoping for something a bit more creative.

Anchor: Creutzfeld-Jakob disease. Mad cow?
House: Mad zebra.

Anchor: Mad zebra. Wernickie's encephalopathy?
House: No, blood thiamine lEvel was normal.

Anchor: I sign your paychecks.
House: I have tenure. Are you going to grab my cane now, stop me from leaving?

Anchor: I thought Everybody lied?
House: Truth begins in lies. Think about it.

Anchor: Patient is orange.
House: The color?

Anchor: No, the fruit.
House: You mean yellow it

In [ ]:
# Проверяем количество полных дубликатов (anchor + line)
duplicate_count = house_df.duplicated(subset=["anchor", "line"]).sum()

# Выбираем и выводим 10 примеров дубликатов
duplicates = house_df[house_df.duplicated(subset=["anchor", "line"], keep=False)]

# Вывод статистики
print(f"Количество дубликатов (одинаковые anchor + line): {duplicate_count}")

# Вывод 10 примеров
print("\n10 примеров дубликатов:\n")
print(duplicates[["anchor", "line"]].head(10))

Количество дубликатов (одинаковые anchor + line): 1

10 примеров дубликатов:

              anchor    line
19169  spoken spoken  spoken
19170  spoken spoken  spoken


In [ ]:
# Удаляем дубликаты по anchor + line, оставляя первую встречающуюся строку
house_df = house_df.drop_duplicates(subset=["anchor", "line"]).reset_index(drop=True)

print("Дубликаты удалены.")

Дубликаты удалены.


In [ ]:
def clean_text_general(text):
    """Удаляет лишние пробелы и исправляет форматирование во всех текстовых колонках."""
    if not isinstance(text, str):
        return ""
    return text.strip()

# Применяем очистку ко всем текстовым колонкам
text_columns = ["anchor", "context_1", "context_2", "context_3", "line"]
for col in text_columns:
    house_df[col] = house_df[col].apply(clean_text_general)

# Проверяем результат на случайных строках
print("\nПримеры очищенных данных:")
print(house_df.sample(5, random_state=42)[text_columns])



Примеры очищенных данных:
                                                  anchor  \
12608           You'd jeopardize a Patient because of my   
20683                                       Let me down.   
12465                                     ... for these.   
1807   So, if you ignore ethics to save one person it...   
20345  No, we already tested his liver enzymes. They'...   

                                               context_1  \
12608           You'd jeopardize a Patient because of my   
20683                                       Let me down.   
12465                                     ... for these.   
1807   So, if you ignore ethics to save one person it...   
20345  No, we already tested his liver enzymes. They'...   

                                               context_2  \
12608  Your conscience bleeds more freely than my hea...   
20683                                          Admit it.   
12465                           I assume you are here...   
1807   What

In [ ]:
total_replicas = len(antagonists_df)
unique_replicas = antagonists_df["line"].nunique()
duplicates_count = total_replicas - unique_replicas

print(f"Всего реплик: {total_replicas}")
print(f"Уникальных реплик: {unique_replicas}")
print(f"Количество повторяющихся реплик: {duplicates_count}")

# Удаление повторяющихся реплик
antagonists_df = antagonists_df.drop_duplicates(subset=["line"]).reset_index(drop=True)

# Проверка
print(f"✅ Размер антагонистов после удаления повторов: {antagonists_df.shape[0]}")



Всего реплик: 167345
Уникальных реплик: 162830
Количество повторяющихся реплик: 4515
✅ Размер антагонистов после удаления повторов: 162830


In [ ]:
# Указываем путь для сохранения
house_cleaned_path = "/content/data/house_extended_cleaned.csv"

antagonists_сleaned_path = "/content/data/antagonists_extended_cleaned.csv"

# Сохранение датасетов
house_df.to_csv(house_cleaned_path, index=False, encoding="utf-8")
antagonists_df.to_csv(antagonists_сleaned_path, index=False, encoding="utf-8")

print(f"Очищенный датасет доктора Хауса сохранён в: {house_cleaned_path}")
print(f"Очищенный датасет антигероев сохранён в: {antagonists_сleaned_path}")

Очищенный датасет доктора Хауса сохранён в: /content/data/house_extended_cleaned.csv
Очищенный датасет антигероев сохранён в: /content/data/antagonists_extended_cleaned.csv


In [ ]:
antagonists_df

,character,line,source
0,WAYMAR ROYCE,What dyou expect? Theyre savages. One lot stea...,Game_of_Thrones
1,WILL,Ive never seen wildlings do a thing like this....,Game_of_Thrones
2,WAYMAR ROYCE,How close did you get?,Game_of_Thrones
3,WILL,Close as any man would.,Game_of_Thrones
4,GARED,We should head back to the wall.,Game_of_Thrones
...,...,...,...
162825,LANDO,I promised to return his ship without a scratc...,Star_Wars
162826,THREEPIO,They did it!,Star_Wars
162827,"""40"" ""LUKE"" ""Greetings, Exalted One. Allow me ...",these two droids.,Star_Wars
162828,"""146"" ""THREEPIO"" ""Victims of the almighty Sarlacc",His Excellency hopes that you will die honorab...,Star_Wars


In [ ]:
house_df

,name,response,season,context_1,context_1_character,context_2,context_2_character,context_3,context_3_character,anchor,anchor_word_count,neg_response
0,house,See that? They all assume I'm a Patient becaus...,season1,"29 year old female, first seizure one month ag...",wilson,"We know that word, the",boy,the,sidney,"29 year old female, first seizure one month ag...",23,This has been a really tough time.
1,house,I don't want them to think I'm a doctor.,season1,So put on a white coat like the rest of us.,wilson,See that? They all assume I'm a Patient becaus...,house,"29 year old female, first seizure one month ag...",wilson,So put on a white coat like the rest of us.,11,Want to come to dinner with us?
2,house,People don't want a sick doctor.,season1,You see where the administration might have a ...,wilson,I don't want them to think I'm a doctor.,house,So put on a white coat like the rest of us.,wilson,You see where the administration might have a ...,12,"Yeah. Oh, OOOH, yeah, you know, did you notice..."
3,house,"The one who can't talk, I liked that part.",season1,Fair enough. I don't like healthy Patients. Th...,wilson,People don't want a sick doctor.,house,You see where the administration might have a ...,wilson,Fair enough. I don't like healthy Patients. Th...,12,"Yes, but I dont think life experience with these."
4,house,And your cousin doesn't like the diagnosis. I ...,season1,She's my cousin.,wilson,"The one who can't talk, I liked that part.",house,Fair enough. I don't like healthy Patients. Th...,wilson,She's my cousin.,3,"Yknow, thanks for trying to cheer me up, but I..."
...,...,...,...,...,...,...,...,...,...,...,...,...
21817,house,Hi.,season8,"Phone. A million times he needed me, and the o...",wilson,House was an ass. That gets Everyone's attenti...,wilson,reading his notes He was my friend. The thing ...,wilson,"Phone. A million times he needed me, and the o...",81,How is this on me?
21818,house,I got out of the back of the building.,season8,How?,wilson,Hi.,house,"Phone. A million times he needed me, and the o...",wilson,Hi. How?,2,Nein! Im greek! My name is Mikanos.
21819,house,Just switched the dental records.,season8,The body,wilson,I got out of the back of the building.,house,How?,wilson,I got out of the back of the building. The body,11,Stop that! Stop that!
21820,house,"I'm dead, Wilson. How do you want to spend you...",season8,You're destroying your entire life. You can't ...,wilson,Just switched the dental records.,house,The body,wilson,You're destroying your entire life. You can't ...,24,I thought the Nights Watch might make a man of...


# Создание триплетов для биэнкодера

In [ ]:
import pandas as pd

# Пути к данным
house_path = "house_extended_cleaned.csv"
antagonists_path = "antagonists_extended_cleaned.csv"

# Загрузка данных
house_df = pd.read_csv(house_path)
antagonists_df = pd.read_csv(antagonists_path)

# Вывод первых строк для проверки
print("Первые строки датасета Доктора Хауса:")
print(house_df.head())

print("Первые строки датасета антагонистов:")
print(antagonists_df.head())

Первые строки датасета Доктора Хауса:
    name                                           response   season  \
0  house  See that? They all assume I'm a Patient becaus...  season1   
1  house           I don't want them to think I'm a doctor.  season1   
2  house                   People don't want a sick doctor.  season1   
3  house         The one who can't talk, I liked that part.  season1   
4  house  And your cousin doesn't like the diagnosis. I ...  season1   

                                           context_1 context_1_character  \
0  29 year old female, first seizure one month ag...              wilson   
1        So put on a white coat like the rest of us.              wilson   
2  You see where the administration might have a ...              wilson   
3  Fair enough. I don't like healthy Patients. Th...              wilson   
4                                   She's my cousin.              wilson   

                                           context_2 context_2_character

In [ ]:
# Создаём датафрейм для триплетов
triplet_df = house_df[["anchor", "response"]].copy()

# Переименовываем колонки для соответствия формату триплетов
triplet_df.rename(columns={"line": "response"}, inplace=True)

# Добавляем пустую колонку для негативного примера
triplet_df["neg_response"] = ""

# Выводим информацию о новом датафрейме
print(f"`triplet_df` сформирован. Размер: {triplet_df.shape}")
print(triplet_df.head(5))

`triplet_df` сформирован. Размер: (21822, 3)
                                              anchor  \
0  29 year old female, first seizure one month ag...   
1        So put on a white coat like the rest of us.   
2  You see where the administration might have a ...   
3  Fair enough. I don't like healthy Patients. Th...   
4                                   She's my cousin.   

                                            response neg_response  
0  See that? They all assume I'm a Patient becaus...               
1           I don't want them to think I'm a doctor.               
2                   People don't want a sick doctor.               
3         The one who can't talk, I liked that part.               
4  And your cousin doesn't like the diagnosis. I ...               


In [ ]:
antagonists_df

,character,line,source
0,WAYMAR ROYCE,What dyou expect? Theyre savages. One lot stea...,Game_of_Thrones
1,WILL,Ive never seen wildlings do a thing like this....,Game_of_Thrones
2,WAYMAR ROYCE,How close did you get?,Game_of_Thrones
3,WILL,Close as any man would.,Game_of_Thrones
4,GARED,We should head back to the wall.,Game_of_Thrones
...,...,...,...
162825,LANDO,I promised to return his ship without a scratc...,Star_Wars
162826,THREEPIO,They did it!,Star_Wars
162827,"""40"" ""LUKE"" ""Greetings, Exalted One. Allow me ...",these two droids.,Star_Wars
162828,"""146"" ""THREEPIO"" ""Victims of the almighty Sarlacc",His Excellency hopes that you will die honorab...,Star_Wars


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Загружаем Sentence Transformer
model = SentenceTransformer("all-mpnet-base-v2")

# Переименовываем "line" в "response"
house_df = house_df.rename(columns={"line": "response"})

# # Векторизуем `anchor` (контекст) и `response` антагонистов
# anchor_embeddings = model.encode(house_df["anchor"].tolist(), convert_to_numpy=True)
# antagonists_embeddings = model.encode(antagonists_df["line"].tolist(), convert_to_numpy=True)

# # Вычисляем косинусное сходство между `anchor` и `line` антагонистов
# similarities = cosine_similarity(anchor_embeddings, antagonists_embeddings)

# Фильтруем антагонистов, исключая слишком короткие реплики
filtered_antagonists = antagonists_df[antagonists_df["line"].apply(lambda x: len(x.split()) >= 3)].reset_index(drop=True)

# Векторизуем `anchor` и `line` антагонистов (уже после фильтрации!)
anchor_embeddings = model.encode(house_df["anchor"].tolist(), convert_to_numpy=True)
antagonists_embeddings = model.encode(filtered_antagonists["line"].tolist(), convert_to_numpy=True)

# Вычисляем косинусное сходство
similarities = cosine_similarity(anchor_embeddings, antagonists_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np
import re

# Очистка `neg_response`
house_df["neg_response"] = ""

# Стандартный вариант: подбор негативных ответов по косинусному сходству с анкором

In [ ]:
# Фильтруем антагонистов: убираем слишком короткие реплики (< 3 слов)
filtered_antagonists = antagonists_df[antagonists_df["line"].apply(lambda x: len(x.split()) >= 3)].reset_index(drop=True)

# Храним использованные негативные ответы
used_negatives = set()
neg_responses = []

# Функция для корректного обрезания реплик
def trim_to_sentence(text, min_words=3):
    """Обрезает текст до первого завершенного предложения (по . ? !), но не короче `min_words`."""
    match = re.search(r"([^.?!]+[.?!])", text)
    if match:
        trimmed_text = match.group(1)
        if len(trimmed_text.split()) >= min_words:
            return trimmed_text
    return text  # Если нет подходящей точки или слишком коротко, берём весь текст

# Перебираем каждую строку house_df
for i, row in house_df.iterrows():
    best_neg_idx = np.argsort(similarities[i])[-20:]  # Берём топ-20 кандидатов
    found = False
    response_len = len(row["response"].split())
    anchor_words = set(row["anchor"].split())  # Слова из `anchor`

    for idx in best_neg_idx[::-1]:  # Идём от самых похожих к менее похожим
        idx = min(idx, len(filtered_antagonists) - 1)  # Предотвращаем выход за границы
        candidate = filtered_antagonists.iloc[idx]["line"]

        # Проверяем, что `neg_response` не входит в `response` и наоборот
        if (
            candidate not in used_negatives and
            candidate not in row["response"] and
            row["response"] not in candidate
        ):
            used_negatives.add(candidate)

            # Обрезаем `neg_response`, сохраняя минимум 3 слова
            candidate = trim_to_sentence(candidate, min_words=3)

            # Проверяем, что `neg_response` не слишком короткий относительно `response`
            neg_len = len(candidate.split())
            if neg_len < max(3, int(response_len * 0.5)):
                continue  # Пропускаем, если слишком короткий

            # Проверяем, не содержит ли `neg_response` ≥ 50% слов из `anchor`
            candidate_words = set(candidate.split())
            common_words = len(anchor_words & candidate_words)
            if common_words / len(anchor_words) > 0.5:
                continue  # Пропускаем, если слишком много общих слов с `anchor`

            neg_responses.append(candidate)
            found = True
            break

    if not found:
        neg_responses.append("")  # Если не нашли, оставляем пустым

# Добавляем `neg_response` в датасет
house_df["neg_response"] = neg_responses

# Проверяем количество заполненных `neg_response`
filled_negatives = house_df["neg_response"].str.len().gt(0).sum()
print(f"Количество заполненных `neg_response`: {filled_negatives}")

# Проверяем, сколько осталось пустых
empty_negatives = house_df["neg_response"].eq("").sum()
print(f"Количество пустых `neg_response`: {empty_negatives}")

# Топ-10 самых частых `neg_response`
top_negatives = house_df["neg_response"].value_counts().head(10)
print("\nТоп-10 самых частых `neg_response`:\n", top_negatives)


Количество заполненных `neg_response`: 18172
Количество пустых `neg_response`: 3650

Топ-10 самых частых `neg_response`:
 neg_response
                       3650
I dont know.             23
You know what?           15
Oh my God!               11
No, no, no.               8
I dont know!              6
I dont think so.          5
What are you doing?       4
Are you kidding?          4
Are you sure?             3
Name: count, dtype: int64


In [ ]:
# Фильтруем только непустые строки
filtered_df = house_df[house_df["neg_response"].str.strip() != ""]

# Проверяем, совпадает ли `neg_response` с `response`
exact_match_count = (filtered_df["neg_response"] == filtered_df["response"]).sum()

# Выводим результат
print(f"Количество `neg_response`, точно совпадающих с `response`: {exact_match_count}")

# Выводим примеры 10 таких строк
exact_matches = filtered_df[filtered_df["neg_response"] == filtered_df["response"]]
print("\nПримеры точных совпадений:")
print(exact_matches[["anchor", "response", "neg_response"]].head(10))

Количество `neg_response`, точно совпадающих с `response`: 0

Примеры точных совпадений:
Empty DataFrame
Columns: [anchor, response, neg_response]
Index: []


In [ ]:
import re
import numpy as np

# Фильтруем строки с пустым `neg_response`
missing_negatives_idx = house_df[house_df["neg_response"].str.strip() == ""].index

print(f"📌 Осталось заполнить {len(missing_negatives_idx)} пустых `neg_response`.")

# Фильтруем антагонистов: убираем слишком короткие реплики (< 2 слов)
available_negatives = antagonists_df[antagonists_df["line"].apply(lambda x: len(x.split()) >= 2)].reset_index(drop=True)

# Функция для корректного обрезания по `.` `?` `!`
def trim_to_sentence(text, min_words=2):
    """Обрезает текст до первого завершенного предложения (по . ? !), но не короче `min_words`."""
    match = re.search(r"([^.?!]+[.?!])", text)
    if match:
        trimmed_text = match.group(1)
        if len(trimmed_text.split()) >= min_words:
            return trimmed_text
    return text  # Если нет подходящей точки или слишком коротко, берём весь текст

# Перемешиваем список доступных антагонистов
shuffled_negatives = available_negatives["line"].sample(frac=1, random_state=42).tolist()
used_negatives = set(house_df["neg_response"].tolist())  # Все уже использованные

# Назначаем оставшиеся `neg_response`
for i, idx in enumerate(missing_negatives_idx):
    if i >= len(shuffled_negatives):
        break  # Если негативных реплик не хватает, оставляем пустые

    candidate = shuffled_negatives[i]

    # Если реплика уже использована 2 раза, ищем другую
    while candidate in used_negatives and list(used_negatives).count(candidate) >= 2 and i < len(shuffled_negatives) - 1:
        i += 1
        candidate = shuffled_negatives[i]

    used_negatives.add(candidate)  # Помечаем как использованный

    # Обрезаем `neg_response` по предложению
    candidate = trim_to_sentence(candidate, min_words=2)

    # Проверяем, что `neg_response` не слишком короткий относительно `response`
    response_len = len(house_df.at[idx, "response"].split())
    if len(candidate.split()) < max(2, int(response_len * 0.5)):
        continue  # Пропускаем, если `neg_response` слишком короткий

    # Проверяем, не содержит ли `neg_response` ≥ 50% слов из `anchor`
    anchor_words = set(house_df.at[idx, "anchor"].split())
    candidate_words = set(candidate.split())
    common_words = len(anchor_words & candidate_words)

    if common_words / len(anchor_words) > 0.5:
        continue  # Пропускаем, если слишком много общих слов с `anchor`

    house_df.at[idx, "neg_response"] = candidate  # Заполняем в датасете

# Проверяем количество обновленных строк
filled_count = house_df["neg_response"].str.strip().gt("").sum()
empty_count = house_df["neg_response"].eq("").sum()

print(f"✅ Всего заполнено `neg_response`: {filled_count}")
print(f"❌ Осталось пустых `neg_response`: {empty_count}")

# Если после повторного прохода остались пустые, можно их дополнительно обработать
if empty_count > 0:
    print("\n🔍 Примеры оставшихся пустых `neg_response` (первые 10):")
    print(house_df[house_df["neg_response"] == ""][["anchor", "response"]].head(10))


📌 Осталось заполнить 3650 пустых `neg_response`.
✅ Всего заполнено `neg_response`: 20005
❌ Осталось пустых `neg_response`: 1817

🔍 Примеры оставшихся пустых `neg_response` (первые 10):
                                                anchor  \
43                 That doesn't mean anything,does it?   
78                                   It's my hospital.   
114                             It's good to meet you.   
165                           What does that tell you?   
188                            Why would you say that?   
255                                I wouldn't do that.   
262                                       Right? Yeah.   
267                           How's your hooker doing?   
286  Don't be condescending. You haven't checked in...   
298                                       No, I don't!   

                                              response  
43   1252 PM Dr. House checks in, please write that...  
78   I did not get her sick, she is not an experime...  
114  

In [ ]:
import random
import numpy as np

# Фильтруем строки с пустым `neg_response`
missing_negatives_idx = house_df[house_df["neg_response"].str.strip() == ""].index

print(f"📌 Осталось заполнить {len(missing_negatives_idx)} пустых `neg_response`.")

# Выбираем `neg_response`, которые встречаются только 1 раз → можно повторно использовать
repeatable_negatives = house_df["neg_response"].value_counts()
repeatable_negatives = repeatable_negatives[repeatable_negatives == 1].index.tolist()

# Перемешиваем, чтобы случайно распределять
random.shuffle(repeatable_negatives)

# Назначаем `neg_response` для оставшихся строк, но не более 2 раз в датасете
assigned_count = {}  # Счётчик использований `neg_response`

for i, idx in enumerate(missing_negatives_idx):
    if i >= len(repeatable_negatives):
        break  # Если негативных реплик не хватает, оставляем пустые

    candidate = repeatable_negatives[i]

    # Проверяем, сколько раз уже использовали
    if assigned_count.get(candidate, 0) < 2:
        # Проверяем, чтобы `neg_response` не был слишком похож на `anchor`
        anchor_text = house_df.at[idx, "anchor"]
        candidate_words = set(candidate.split())
        anchor_words = set(anchor_text.split())

        # Косинусное сходство anchor ↔ neg_response (если векторы уже есть)
        if "anchor_neg_sim" in house_df.columns:
            similarity = house_df.at[idx, "anchor_neg_sim"]
            if similarity > 0.3:  # Пропускаем, если слишком похож
                continue

        # Проверяем, не содержит ли `neg_response` ≥ 50% слов из `anchor`
        common_words = len(anchor_words & candidate_words)
        if common_words / len(anchor_words) > 0.5:
            continue  # Пропускаем, если слишком много общих слов с `anchor`

        house_df.at[idx, "neg_response"] = candidate  # Заполняем в датасете
        assigned_count[candidate] = assigned_count.get(candidate, 0) + 1  # Увеличиваем счётчик

# Проверяем количество обновленных строк
filled_count = house_df["neg_response"].str.strip().gt("").sum()
empty_count = house_df["neg_response"].eq("").sum()

print(f"✅ Всего заполнено `neg_response`: {filled_count}")
print(f"❌ Осталось пустых `neg_response`: {empty_count}")


📌 Осталось заполнить 1817 пустых `neg_response`.
✅ Всего заполнено `neg_response`: 21820
❌ Осталось пустых `neg_response`: 2


In [ ]:
# Найти индекс последней пустой строки
empty_idx = house_df[house_df["neg_response"].str.strip() == ""].index

# Вывести эту строку
print("\n🔍 Единственная пустая `neg_response`:")
print(house_df.loc[empty_idx, ["anchor", "response"]])


🔍 Единственная пустая `neg_response`:
                         anchor  \
7752  What are you going to do?   
9068             I know it now.   

                                               response  
7752  I thought I'd get your theories, mock them, th...  
9068  Your mind convinced your body to get a rash, p...  


In [ ]:
# Удаляем строку, где `neg_response` пустой
house_df = house_df[house_df["neg_response"].str.strip() != ""]

# Проверяем, что строк больше нет
empty_count = house_df["neg_response"].eq("").sum()
print(f"✅ Осталось пустых `neg_response`: {empty_count}")

# Сохранение обновленного датасета
house_df.to_csv("/content/cleaned_triplet_data.csv", index=False, encoding="utf-8-sig")
print("📁 Обновленный датасет сохранен: `cleaned_triplet_data.csv`")

✅ Осталось пустых `neg_response`: 0
📁 Обновленный датасет сохранен: `cleaned_triplet_data.csv`


In [ ]:
# Подсчёт количества повторений `neg_response`
neg_response_counts = house_df["neg_response"].value_counts()

# Количество уникальных `neg_response`
unique_neg_count = neg_response_counts.nunique()

# Найдём `neg_response`, которые встречаются более 2 раз
duplicates = neg_response_counts[neg_response_counts > 2]

print(f"Всего уникальных `neg_response`: {unique_neg_count}")
print(f"Количество `neg_response`, встречающихся более 2 раз: {len(duplicates)}")

# Выводим примеры повторяющихся `neg_response`
print("\nТоп-10 самых часто повторяющихся `neg_response`:")
print(duplicates.head(10))

Всего уникальных `neg_response`: 10
Количество `neg_response`, встречающихся более 2 раз: 24

Топ-10 самых часто повторяющихся `neg_response`:
neg_response
I dont know.                   26
You know what?                 16
Oh my God!                     12
No, no, no.                     9
I dont know!                    6
I dont think so.                6
Are you kidding?                5
What are you talking about?     5
What are you doing?             4
Oh my God.                      3
Name: count, dtype: int64


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Проверяем, какие embeddings уже есть
print("Проверка наличия embeddings:")
print("anchor_embeddings" in locals())  # Уже есть
print("antagonists_embeddings" in locals())  # Уже есть
print("response_embeddings" in locals())  # Отсутствует
print("neg_response_embeddings" in locals())  # Отсутствует

# Векторизуем `response`, если он ещё не векторизован
if "response_embeddings" not in locals():
    print("Векторизация `response`...")
    response_embeddings = model.encode(house_df["response"].tolist(), convert_to_numpy=True)

# Векторизуем `neg_response`, если он ещё не векторизован
if "neg_response_embeddings" not in locals():
    print("Векторизация `neg_response`...")
    neg_response_embeddings = model.encode(house_df["neg_response"].tolist(), convert_to_numpy=True)

# Считаем косинусное сходство
anchor_response_sim = cosine_similarity(anchor_embeddings, response_embeddings).diagonal()
anchor_neg_sim = cosine_similarity(anchor_embeddings, neg_response_embeddings).diagonal()
response_neg_sim = cosine_similarity(response_embeddings, neg_response_embeddings).diagonal()

# Выводим средние значения сходства
print(f"Средняя векторная близость:")
print(f"`anchor` ↔ `response`: {np.mean(anchor_response_sim):.4f}")
print(f"`anchor` ↔ `neg_response`: {np.mean(anchor_neg_sim):.4f}")
print(f"`response` ↔ `neg_response`: {np.mean(response_neg_sim):.4f}")

Проверка наличия embeddings:
True
True
False
False
Векторизация `response`...
Векторизация `neg_response`...
Средняя векторная близость:
`anchor` ↔ `response`: 0.1637
`anchor` ↔ `neg_response`: 0.2421
`response` ↔ `neg_response`: 0.1548


Мне показались бы результаты векторной близости необычными, если бы я впервые работала с этим датасетом. Но доктор Хаус настолько нестандартный герой, что иного результата я и не ждала...

anchor ↔ neg_response (0.3904) → выше, чем anchor ↔ response (0.2169).

Обычно ожидается, что anchor ↔ response будет самым высоким.
Проблема: neg_response оказывается ближе к anchor, чем response.
Это означает, что подбор neg_response мог быть неидеальным и стоит пересмотреть его критерии.

response ↔ neg_response (0.1449) → это хорошее значение.

Оно самое низкое, что правильно.
Это значит, что neg_response действительно отличается от response, что соответствует задаче.

In [ ]:
too_similar_idx = np.where(anchor_neg_sim > anchor_response_sim)[0]
print("\nПримеры, где `neg_response` ближе к `anchor`, чем `response`:")
print(house_df.iloc[too_similar_idx][["anchor", "response", "neg_response"]].head(10))


Примеры, где `neg_response` ближе к `anchor`, чем `response`:
                                               anchor  \
0   29 year old female, first seizure one month ag...   
1         So put on a white coat like the rest of us.   
2   You see where the administration might have a ...   
3   Fair enough. I don't like healthy Patients. Th...   
4                                    She's my cousin.   
6   Hah, yes, just a simple country doctor. Brain ...   
7   Protein markers for the three most prevalent b...   
8                                  No family history.   
9               Other side. No environmental factors.   
10   And she's not responding to radiation treatment.   

                                             response  \
0   See that? They all assume I'm a Patient becaus...   
1            I don't want them to think I'm a doctor.   
2                    People don't want a sick doctor.   
3          The one who can't talk, I liked that part.   
4   And your cousin does

In [ ]:
# Пути для сохранения файлов
triplet_file = "/content/triplet_extended_data.csv"
anchor_embeddings_file = "/content/anchor_extended_embeddings.npy"
antagonists_embeddings_file = "/content/antagonists_extended_embeddings.npy"

# Сохраняем обновленный триплетный датасет
house_df[["anchor", "response", "neg_response"]].to_csv(triplet_file, index=False, encoding="utf-8")
print(f"Файл `triplet_extended_data.csv` сохранён: {triplet_file}")

# Сохраняем векторизованные списки
np.save(anchor_embeddings_file, anchor_embeddings)
np.save(antagonists_embeddings_file, antagonists_embeddings)

print(f"Файл `anchor_extended_embeddings.npy` сохранён: {anchor_embeddings_file}")
print(f"Файл `antagonists_extended_embeddings.npy` сохранён: {antagonists_embeddings_file}")

Файл `triplet_extended_data.csv` сохранён: /content/triplet_extended_data.csv
Файл `anchor_extended_embeddings.npy` сохранён: /content/anchor_extended_embeddings.npy
Файл `antagonists_extended_embeddings.npy` сохранён: /content/antagonists_extended_embeddings.npy


In [ ]:
import pickle

# Пути для сохранения файлов
triplets_path =  "/content/data/triplets_extended.pkl"

triplets_extended_df = house_df[["anchor", "response", "neg_response"]]

# Проверяем, что в триплетах нет NaN перед сохранением
assert triplets_extended_df.isna().sum().sum() == 0, "Ошибка: В триплетах есть NaN!"

# Сохранение триплетов в pickle-формате
with open(triplets_path, "wb") as f:
    pickle.dump(triplets_extended_df, f)

print(f"Триплеты сохранены в: {triplets_path}")

# Автоматическая загрузка файлов для скачивания
files.download(triplets_path)

Триплеты сохранены в: /content/data/triplets_extended.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Гипотеза 1: Выбирать neg_response, которые не превышают порог  < 0.18 по косинусному сходству.

Реплики с косинусным сходством выше 0.18 слишком похожи на anchor.
Реплики с косинусным сходством ниже 0.1 слишком далеки и могут быть неуместными, что ухудшит качество модели.
Этот баланс должен обеспечить хорошее качество обучения.

In [ ]:
import numpy as np
import re
from collections import defaultdict

# Очистка neg_response перед новым запуском
house_df["neg_response"] = ""

# Счётчик повторов негативных ответов
negative_usage_counter = defaultdict(int)

# Список для негативных ответов
neg_responses = []

# Перебор каждой строки house_df
for i, row in house_df.iterrows():
    sorted_neg_idx = np.argsort(similarities[i])[::-1]

    found = False
    anchor_words = set(row["anchor"].split())
    response_len = len(row["response"].split())

    # Перебираем индексы кандидатов
    for idx in sorted_neg_idx:
        similarity_score = similarities[i, idx]

        # Используем строго заданный диапазон (0.1 - 0.18)
        if similarity_score < 0.1 or similarity_score > 0.18:
            continue

        candidate = filtered_antagonists.iloc[idx]["line"]

        # Проверка повторов (не более 2 раз)
        if negative_usage_counter[candidate] >= 2:
            continue

        # Проверка совпадения с response
        if candidate in row["response"] or row["response"] in candidate:
            continue

        candidate = candidate.strip()

        # Проверка длины
        neg_len = len(candidate.split())
        if neg_len < max(3, int(response_len * 0.5)):
            continue

        # Проверка пересечения слов с anchor
        candidate_words = set(candidate.split())
        common_words_ratio = len(anchor_words & candidate_words) / len(anchor_words)

        if common_words_ratio > 0.5:
            continue

        # Если прошли проверки, сохраняем негатив
        neg_responses.append(candidate)
        negative_usage_counter[candidate] += 1
        found = True
        break

    if not found:
        neg_responses.append("")  # Если не нашли, оставляем пустым

# Заполняем датасет
house_df["neg_response"] = neg_responses

# Проверка результатов
filled_negatives = house_df["neg_response"].str.strip().gt("").sum()
empty_negatives = house_df["neg_response"].eq("").sum()

print(f"Количество заполненных neg_response: {filled_negatives}")
print(f"Количество пустых neg_response: {empty_negatives}")

# Проверяем повторы
top_negatives = house_df["neg_response"].value_counts().head(10)
print("\nТоп-10 самых частых neg_response:\n", top_negatives)

if empty_negatives > 0:
    print("\nПервые 10 пустых примеров neg_response:")
    print(house_df[house_df["neg_response"].eq("")][["anchor", "response"]].head(10))


Количество заполненных neg_response: 21822
Количество пустых neg_response: 0

Топ-10 самых частых neg_response:
 neg_response
Littlefingers not in love with me.                                                                   2
Not now Dwight, We got a call about a disturbance?                                                   2
Did you umIm sorry, did you just say its Rachel?                                                     2
I thought I was helping you. I expected a hug. I took care of that cat, the best way I knew how.     2
Good idea. And since youve had such a rough day, Im gonna let you throw the pie in Leonards face.    2
Well, gotta go.                                                                                      2
Her ass print is still on your grandmothers quilt, do you really want to talk about smoking?         2
Well, I was probably going to do it at some point.                                                   2
You were on set, you saw what it was like.        

In [ ]:
# Подсчитываем частоту встречаемости всех `neg_response`
neg_counts = house_df["neg_response"].value_counts()

# Отбираем те, которые встречаются ровно 2 раза
neg_repeated_twice = neg_counts[neg_counts == 2]

# Выводим количество таких повторов
print(f"Количество `neg_response`, встречающихся ровно 2 раза: {len(neg_repeated_twice)}")

# Выводим первые 10 примеров
print("\nПримеры `neg_response`, повторяющихся ровно 2 раза:")
print(neg_repeated_twice.head(10))


Количество `neg_response`, встречающихся ровно 2 раза: 1919

Примеры `neg_response`, повторяющихся ровно 2 раза:
neg_response
Littlefingers not in love with me.                                                                   2
Not now Dwight, We got a call about a disturbance?                                                   2
Did you umIm sorry, did you just say its Rachel?                                                     2
I thought I was helping you. I expected a hug. I took care of that cat, the best way I knew how.     2
Good idea. And since youve had such a rough day, Im gonna let you throw the pie in Leonards face.    2
Well, gotta go.                                                                                      2
Her ass print is still on your grandmothers quilt, do you really want to talk about smoking?         2
Well, I was probably going to do it at some point.                                                   2
You were on set, you saw what it was like.        

In [ ]:
import numpy as np
import re

# Пересчитываем сходство между anchor и neg_response
anchor_neg_sim = np.array([
    cosine_similarity(
        anchor_embeddings[i].reshape(1, -1),
        model.encode(house_df.iloc[i]["neg_response"]).reshape(1, -1)
    )[0, 0]
    for i in range(len(house_df))
])

# Добавляем это сходство в датафрейм
house_df["anchor_neg_sim"] = anchor_neg_sim

# Подсчитываем частоту `neg_response`
neg_counts = house_df["neg_response"].value_counts()

# Выбираем `neg_response`, которые встречаются ровно 2 раза
neg_repeated_twice = neg_counts[neg_counts == 2].index.tolist()

# Разделяем пары: 1 сохраняем, 1 заменяем
to_keep = []
to_replace = []

for neg in neg_repeated_twice:
    indexes = house_df[house_df["neg_response"] == neg].index.tolist()

    if len(indexes) == 2:
        # Оставляем тот, который ближе к `anchor`, заменяем второй
        idx1, idx2 = indexes
        if house_df.at[idx1, "anchor_neg_sim"] > house_df.at[idx2, "anchor_neg_sim"]:
            to_keep.append(idx1)
            to_replace.append(idx2)
        else:
            to_keep.append(idx2)
            to_replace.append(idx1)

print(f"✅ Оставляем без изменений: {len(to_keep)} `neg_response` (наиболее близкие к `anchor`)")
print(f"🔄 Перезаполняем: {len(to_replace)} `neg_response`")

# Перезаписываем `neg_response` для тех, что в `to_replace`
replace_counter = 0
for idx in to_replace:
    sorted_neg_idx = np.argsort(similarities[idx])[::-1]  # Сортируем по убыванию схожести

    found = False
    anchor_words = set(house_df.at[idx, "anchor"].split())

    for new_idx in sorted_neg_idx:
        new_similarity = similarities[idx, new_idx]

        # Подбираем `neg_response` с близостью 0.07–0.099
        if not (0.07 <= new_similarity <= 0.099):
            continue

        new_candidate = filtered_antagonists.iloc[new_idx]["line"]

        # Проверяем, что кандидат ещё не использовался
        if new_candidate in house_df["neg_response"].values:
            continue

        # Проверяем, что кандидат не входит в `response`
        if new_candidate in house_df.at[idx, "response"] or house_df.at[idx, "response"] in new_candidate:
            continue

        # Проверяем, что новый кандидат не слишком похож на `anchor`
        new_words = set(new_candidate.split())
        common_words_ratio = len(anchor_words & new_words) / len(anchor_words)
        if common_words_ratio > 0.5:
            continue

        # Если всё хорошо, заменяем `neg_response`
        house_df.at[idx, "neg_response"] = new_candidate
        replace_counter += 1
        found = True
        break

    if not found:
        house_df.at[idx, "neg_response"] = ""  # Если ничего не нашли, оставляем пустым

print(f"✅ Перезаполнено {replace_counter} `neg_response`, заменяя менее подходящие пары.")

# Проверяем итоговые повторения
updated_neg_counts = house_df["neg_response"].value_counts()
updated_repeated_twice = updated_neg_counts[updated_neg_counts == 2]

print(f"\nПосле исправления, `neg_response`, встречающихся ровно 2 раза: {len(updated_repeated_twice)}")

# Выводим примеры исправленных `neg_response`
print("\n🔍 Примеры исправленных `neg_response`:")
print(house_df.iloc[to_replace][["anchor", "response", "neg_response"]].head(10))


✅ Оставляем без изменений: 1919 `neg_response` (наиболее близкие к `anchor`)
🔄 Перезаполняем: 1919 `neg_response`
✅ Перезаполнено 1919 `neg_response`, заменяя менее подходящие пары.

После исправления, `neg_response`, встречающихся ровно 2 раза: 0

🔍 Примеры исправленных `neg_response`:
                                                  anchor  \
4461                                    Take five. Stop.   
5167                     Maybe she was having an affair.   
1329   Her temperature's 105. Treatment's not working...   
4008                            Now why would I do that?   
13908               Same answer now you owe her a favor.   
15835  You want her to stay because you want her back...   
19820           You alright? You go to lunch with Cuddy?   
17417  I didn't know there was a section of the brain...   
12332                 Welcome back. I'm Nurse Dickerson.   
6805                A lot of people dictate their notes.   

                                                res

In [ ]:
# Подсчитываем частоту встречаемости всех `neg_response`
neg_counts = house_df["neg_response"].value_counts()

# Фильтруем `neg_response`, которые встречаются более 1 раза
neg_more_than_once = neg_counts[neg_counts > 1]

print(f"Количество `neg_response`, встречающихся более 1 раза: {len(neg_more_than_once)}")

# Выводим первые 10 примеров
print("\nПримеры `neg_response`, повторяющихся более 1 раза:")
print(neg_more_than_once.head(10))

Количество `neg_response`, встречающихся более 1 раза: 0

Примеры `neg_response`, повторяющихся более 1 раза:
Series([], Name: count, dtype: int64)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Проверяем, какие embeddings уже есть
print("Проверка наличия embeddings:")
print("anchor_embeddings" in locals())  # Уже есть
print("antagonists_embeddings" in locals())  # Уже есть
print("response_embeddings" in locals())  # Отсутствует
print("neg_response_embeddings" in locals())  # Отсутствует

# Векторизуем `response`, если он ещё не векторизован
if "response_embeddings" not in locals():
    print("Векторизация `response`...")
    response_embeddings = model.encode(house_df["response"].tolist(), convert_to_numpy=True)

# Векторизуем `neg_response`, если он ещё не векторизован
if "neg_response_embeddings" not in locals():
    print("Векторизация `neg_response`...")
    neg_response_embeddings = model.encode(house_df["neg_response"].tolist(), convert_to_numpy=True)

# Считаем косинусное сходство
anchor_response_sim = cosine_similarity(anchor_embeddings, response_embeddings).diagonal()
anchor_neg_sim = cosine_similarity(anchor_embeddings, neg_response_embeddings).diagonal()
response_neg_sim = cosine_similarity(response_embeddings, neg_response_embeddings).diagonal()

# Выводим средние значения сходства
print(f"Средняя векторная близость:")
print(f"`anchor` ↔ `response`: {np.mean(anchor_response_sim):.4f}")
print(f"`anchor` ↔ `neg_response`: {np.mean(anchor_neg_sim):.4f}")
print(f"`response` ↔ `neg_response`: {np.mean(response_neg_sim):.4f}")

Проверка наличия embeddings:
True
True
False
False
Векторизация `response`...
Векторизация `neg_response`...
Средняя векторная близость:
`anchor` ↔ `response`: 0.2169
`anchor` ↔ `neg_response`: 0.1729
`response` ↔ `neg_response`: 0.1176


In [84]:
import pickle

# Пути для сохранения файлов
triplet_file = "/content/triplet_limited_cos.csv"
anchor_embeddings_file = "/content/anchor_limited_cos_embeddings.npy"
antagonists_embeddings_file = "/content/antagonists_limited_cos_embeddings.npy"
triplets_path =  "/content/data/triplets_limited_cos_extended1.pkl"

# Сохраняем обновленный триплетный датасет
house_df[["anchor", "response", "neg_response"]].to_csv(triplet_file, index=False, encoding="utf-8")
print(f"Файл триплетов в .csv сохранён: {triplet_file}")

# Сохраняем векторизованные списки
np.save(anchor_embeddings_file, anchor_embeddings)
np.save(antagonists_embeddings_file, antagonists_embeddings)

print(f"Файл .npy сохранён: {anchor_embeddings_file}")
print(f"Файл антигерои в векторах сохранён: {antagonists_embeddings_file}")

triplets_extended_df = house_df[["anchor", "response", "neg_response"]]

# Проверяем, что в триплетах нет NaN перед сохранением
assert triplets_extended_df.isna().sum().sum() == 0, "Ошибка: В триплетах есть NaN!"

# Сохранение триплетов в pickle-формате
with open(triplets_path, "wb") as f:
    pickle.dump(triplets_extended_df, f)

print(f"Триплеты сохранены в: {triplets_path}")

# Автоматическая загрузка файлов для скачивания
files.download(triplets_path)

Файл триплетов в .csv сохранён: /content/triplet_limited_cos.csv
Файл .npy сохранён: /content/anchor_limited_cos_embeddings.npy
Файл антигерои в векторах сохранён: /content/antagonists_limited_cos_embeddings.npy
Триплеты сохранены в: /content/data/triplets_limited_cos_extended1.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
# Создаем датасет для кросс-энкодера
cross_encoder_data = pd.DataFrame()

# Позитивные пары (метка `1`)
cross_encoder_data["input_text"] = house_df["anchor"] + " [SEP] " + house_df["response"]
cross_encoder_data["label"] = 1

# Негативные пары (метка `0`)
neg_data = pd.DataFrame()
neg_data["input_text"] = house_df["anchor"] + " [SEP] " + house_df["neg_response"]
neg_data["label"] = 0

# Объединяем позитивные и негативные примеры
cross_encoder_data = pd.concat([cross_encoder_data, neg_data], ignore_index=True)

# Сохраняем в CSV
cross_encoder_data.to_csv("/content/cross_encoder_data_limited_cos1.csv", index=False, encoding="utf-8-sig")

print("📁 Файл `cross_encoder_data.csv` сохранен.")

# Пути для сохранения файлов
reranker_extended_path =  "/content/data/reranker_limited_cos1.pkl"

# Проверяем, что в датасете нет NaN перед сохранением
assert cross_encoder_data.isna().sum().sum() == 0, "Ошибка: В данных есть NaN!"

# Сохранение датасета в pickle-формате
with open(reranker_extended_path, "wb") as f:
    pickle.dump(cross_encoder_data, f)

print(f"Датасет для реренкера сохранен в: {reranker_extended_path}")

# Автоматическая загрузка файлов для скачивания
files.download(reranker_extended_path)

📁 Файл `cross_encoder_data.csv` сохранен.
Датасет для реренкера сохранен в: /content/data/reranker_limited_cos1.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Гипотеза 2: Выбирать neg_response не из топ-20 самых похожих, а из диапазона от топ-30 до топ-100

Выбор негативных кандидатов происходит не среди топ-20, а из диапазона от 30 до 100.
Это гарантирует, что выбранные негативные реплики будут менее похожими на anchor, тем самым снижая вероятность близости выше позитивных примеров.
Сохранились проверки на количество повторов (не более 2 раз), длину реплики и пересечение слов с anchor.

In [ ]:
import numpy as np
import re

# Очистка neg_response перед новым заполнением
house_df["neg_response"] = ""

# Фильтрация антагонистов: только >= 3 слов
filtered_antagonists = antagonists_df[
    antagonists_df["line"].apply(lambda x: len(x.split()) >= 3)
].reset_index(drop=True)

# Функция обрезки реплик по предложению
def trim_to_sentence(text, min_words=3):
    match = re.search(r"([^.?!]+[.?!])", text)
    if match:
        trimmed_text = match.group(1)
        if len(trimmed_text.split()) >= min_words:
            return trimmed_text
    return text

used_negatives = {}
neg_responses = []

# Заполняем neg_response заново по гипотезе №2
for i, row in house_df.iterrows():
    sorted_neg_idx = np.argsort(similarities[i])[::-1]

    anchor_words = set(row["anchor"].split())
    response_len = len(row["response"].split())
    found = False

    # Берём кандидатов из диапазона от 30 до 100 по сходству
    candidate_indices = sorted_neg_idx[30:100]

    for idx in candidate_indices:
        candidate = filtered_antagonists.iloc[idx]["line"]

        # Проверка повторений (не больше 2 раз)
        if used_negatives.get(candidate, 0) >= 2:
            continue

        # Проверка на совпадения с response
        if candidate in row["response"] or row["response"] in candidate:
            continue

        # Обрезаем по предложению
        candidate = trim_to_sentence(candidate, min_words=3)

        # Проверка длины негатива
        neg_len = len(candidate.split())
        if neg_len < max(3, int(response_len * 0.5)):
            continue

        # Проверка пересечения слов с anchor
        candidate_words = set(candidate.split())
        common_words_ratio = len(anchor_words & candidate_words) / len(anchor_words)
        if common_words_ratio > 0.5:
            continue

        # Заполняем и отмечаем использованным
        neg_responses.append(candidate)
        used_negatives[candidate] = used_negatives.get(candidate, 0) + 1
        found = True
        break

    if not found:
        neg_responses.append("")

# Добавляем neg_response в датасет
house_df["neg_response"] = neg_responses

# Проверка результатов
filled_negatives = house_df["neg_response"].str.strip().gt("").sum()
empty_negatives = house_df["neg_response"].eq("").sum()

print(f"Количество заполненных neg_response: {filled_negatives}")
print(f"Количество пустых neg_response: {empty_negatives}")

# Топ-10 самых частых neg_response
top_negatives = house_df["neg_response"].value_counts().head(10)
print("\nТоп-10 самых частых neg_response:\n", top_negatives)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Проверяем, какие embeddings уже есть
print("Проверка наличия embeddings:")
print("anchor_embeddings" in locals())  # Уже есть
print("antagonists_embeddings" in locals())  # Уже есть
print("response_embeddings" in locals())  # Отсутствует
print("neg_response_embeddings" in locals())  # Отсутствует

# Векторизуем `response`, если он ещё не векторизован
if "response_embeddings" not in locals():
    print("Векторизация `response`...")
    response_embeddings = model.encode(house_df["response"].tolist(), convert_to_numpy=True)

# Векторизуем `neg_response`, если он ещё не векторизован
if "neg_response_embeddings" not in locals():
    print("Векторизация `neg_response`...")
    neg_response_embeddings = model.encode(house_df["neg_response"].tolist(), convert_to_numpy=True)

# Считаем косинусное сходство
anchor_response_sim = cosine_similarity(anchor_embeddings, response_embeddings).diagonal()
anchor_neg_sim = cosine_similarity(anchor_embeddings, neg_response_embeddings).diagonal()
response_neg_sim = cosine_similarity(response_embeddings, neg_response_embeddings).diagonal()

# Выводим средние значения сходства
print(f"Средняя векторная близость:")
print(f"`anchor` ↔ `response`: {np.mean(anchor_response_sim):.4f}")
print(f"`anchor` ↔ `neg_response`: {np.mean(anchor_neg_sim):.4f}")
print(f"`response` ↔ `neg_response`: {np.mean(response_neg_sim):.4f}")

In [ ]:
import pickle

# Пути для сохранения файлов
triplet_file = "/content/triplet_top_100.csv"
anchor_embeddings_file = "/content/anchor_top_100_embeddings.npy"
antagonists_embeddings_file = "/content/antagonists_top_100_embeddings.npy"
triplets_path =  "/content/data/triplets_top_100_extended.pkl"

# Сохраняем обновленный триплетный датасет
house_df[["anchor", "response", "neg_response"]].to_csv(triplet_file, index=False, encoding="utf-8")
print(f"Файл триплетов в .csv сохранён: {triplet_file}")

# Сохраняем векторизованные списки
np.save(anchor_embeddings_file, anchor_embeddings)
np.save(antagonists_embeddings_file, antagonists_embeddings)

print(f"Файл .npy сохранён: {anchor_embeddings_file}")
print(f"Файл антигерои в векторах сохранён: {antagonists_embeddings_file}")

triplets_extended_df = house_df[["anchor", "response", "neg_response"]]

# Проверяем, что в триплетах нет NaN перед сохранением
assert triplets_extended_df.isna().sum().sum() == 0, "Ошибка: В триплетах есть NaN!"

# Сохранение триплетов в pickle-формате
with open(triplets_path, "wb") as f:
    pickle.dump(triplets_extended_df, f)

print(f"Триплеты сохранены в: {triplets_path}")

# Автоматическая загрузка файлов для скачивания
files.download(triplets_path)

## **Финальная структура обучающего датасета для би-энкодера**

### **Описание полей**
| Колонка        | Описание  | Пример |
|---------------|----------|--------|
| **`anchor`** | Контекст перед ответом | `"29 year old female, first seizure one month ago. We know that word, the"` |
| **`response`** | Правильный ответ (реплика Доктора Хауса) | `"See that? They all assume I'm a Patient because of the cane."` |
| **`neg_response`** | Негативный ответ, выбранный по векторной близости | `"Cut. Yikes, this guy is worse than Wil Wheaton."` |

---

### **Логика формирования `neg_response`**
1. **Векторизуем `anchor` и `line` антагонистов**.  
2. **Ищем `neg_response`, максимально похожий на `response`, но неверный**.  
3. **Фильтруем слишком короткие реплики** (не менее 3 слов).  
4. **Подбираем `neg_response`, который не короче 75% `response`**.  
5. **Обрезаем `neg_response` по первому `.` `?` `!`, но не короче 3 слов**.  
6. **Исключаем совпадения `neg_response` == `response`**, а также случаи, когда `response` входит в `neg_response` и наоборот.  

---

### **Дополнительные правила обработки данных**
- **`anchor` содержит контекст**, который может включать **последнее предложение из `context_2`**, если `context_1` слишком короткий.  
- **`response` — это истинный ответ**, который даёт Доктор Хаус.  
- **`neg_response` подбирается так, чтобы он выглядел правдоподобным, но был неправильным ответом**.  
- **Если не найден осмысленный `neg_response`, он остаётся пустым** (в дальнейшем можно дополнительно обработать).  

Этот формат предназначен для **обучения би-энкодера**, который должен находить наиболее релевантный ответ на `anchor`.


# Гипотеза 3: три негативных ответа вместо одного

In [ ]:
import numpy as np
import re
import pandas as pd

# Фильтрация реплик антагонистов (>= 3 слов)
filtered_antagonists = antagonists_df[
    antagonists_df["line"].apply(lambda x: len(x.split()) >= 3)
].reset_index(drop=True)

# Функция обрезки реплик по предложению
def trim_to_sentence(text, min_words=3):
    match = re.search(r"([^.?!]+[.?!])", text)
    if match:
        trimmed_text = match.group(1)
        if len(trimmed_text.split()) >= min_words:
            return trimmed_text
    return text

# Хранение использования негативов
used_negatives = {}

# Списки для негативных ответов
neg_responses_1 = []
neg_responses_2 = []
neg_responses_3 = []

# Перебор каждой строки
for i, row in house_df.iterrows():
    sorted_neg_idx = np.argsort(similarities[i])[::-1]  # Сортируем по убыванию схожести

    anchor_words = set(row["anchor"].split())
    response_len = len(row["response"].split())

    selected_negs = []

    for idx in sorted_neg_idx:
        similarity_score = similarities[i, idx]
        candidate = filtered_antagonists.iloc[idx]["line"]

        # Проверка количества повторений (не больше 2 раз)
        if used_negatives.get(candidate, 0) >= 2:
            continue

        # Проверка совпадений с response и уже выбранными негативами
        if (
            candidate in row["response"]
            or row["response"] in candidate
            or candidate in selected_negs
        ):
            continue

        candidate = trim_to_sentence(candidate, min_words=3)

        neg_len = len(candidate.split())
        if neg_len < max(3, int(response_len * 0.5)):
            continue

        candidate_words = set(candidate.split())
        common_words_ratio = len(anchor_words & candidate_words) / len(anchor_words)
        if common_words_ratio > 0.5:
            continue

        # Разделяем негативы по косинусному сходству
        if 0.10 <= similarity_score <= 0.18 and len(selected_negs) == 0:
            selected_negs.append(candidate)  # `neg_response_1` (hard negative)
        elif 0.07 <= similarity_score <= 0.099 and len(selected_negs) < 3:
            selected_negs.append(candidate)  # `neg_response_2` и `neg_response_3`

        used_negatives[candidate] = used_negatives.get(candidate, 0) + 1

        if len(selected_negs) == 3:
            break  # Нашли все 3 негативных примера

    # Если найдено меньше 3 негативных, дополняем пустыми строками
    while len(selected_negs) < 3:
        selected_negs.append("")

    neg_responses_1.append(selected_negs[0])
    neg_responses_2.append(selected_negs[1])
    neg_responses_3.append(selected_negs[2])

# Обновляем датасет
house_df["neg_response_1"] = neg_responses_1
house_df["neg_response_2"] = neg_responses_2
house_df["neg_response_3"] = neg_responses_3

# Проверка результатов
for n in range(1, 4):
    filled_negatives = house_df[f"neg_response_{n}"].str.strip().gt("").sum()
    empty_negatives = house_df[f"neg_response_{n}"].eq("").sum()
    print(f"neg_response_{n}: заполнено {filled_negatives}, пустых {empty_negatives}")

# Проверка повторов самых частых негативов
top_negatives = pd.concat([
    house_df["neg_response_1"],
    house_df["neg_response_2"],
    house_df["neg_response_3"]
]).value_counts().head(10)

print("\nТоп-10 самых частых neg_response во всех негативных колонках:\n", top_negatives)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Проверяем, какие embeddings уже есть
print("Проверка наличия embeddings:")
print("anchor_embeddings" in locals())  # Уже есть
print("antagonists_embeddings" in locals())  # Уже есть
print("response_embeddings" in locals())  # Отсутствует
print("neg_response_embeddings" in locals())  # Отсутствует

# Векторизуем `response`, если он ещё не векторизован
if "response_embeddings" not in locals():
    print("Векторизация `response`...")
    response_embeddings = model.encode(house_df["response"].tolist(), convert_to_numpy=True)

# Векторизуем `neg_response`, если он ещё не векторизован
if "neg_response_embeddings" not in locals():
    print("Векторизация `neg_response`...")
    neg_response_embeddings = model.encode(house_df["neg_response"].tolist(), convert_to_numpy=True)

# Считаем косинусное сходство
anchor_response_sim = cosine_similarity(anchor_embeddings, response_embeddings).diagonal()
anchor_neg_sim = cosine_similarity(anchor_embeddings, neg_response_embeddings).diagonal()
response_neg_sim = cosine_similarity(response_embeddings, neg_response_embeddings).diagonal()

# Выводим средние значения сходства
print(f"Средняя векторная близость:")
print(f"`anchor` ↔ `response`: {np.mean(anchor_response_sim):.4f}")
print(f"`anchor` ↔ `neg_response`: {np.mean(anchor_neg_sim):.4f}")
print(f"`response` ↔ `neg_response`: {np.mean(response_neg_sim):.4f}")

In [ ]:
import pickle

# Пути для сохранения файлов
triplet_file = "/content/triplet_top_100.csv"
anchor_embeddings_file = "/content/anchor_top_100_embeddings.npy"
antagonists_embeddings_file = "/content/antagonists_top_100_embeddings.npy"
triplets_path =  "/content/data/triplets_top_100_extended.pkl"

# Сохраняем обновленный триплетный датасет
house_df[["anchor", "response", "neg_response"]].to_csv(triplet_file, index=False, encoding="utf-8")
print(f"Файл триплетов в .csv сохранён: {triplet_file}")

# Сохраняем векторизованные списки
np.save(anchor_embeddings_file, anchor_embeddings)
np.save(antagonists_embeddings_file, antagonists_embeddings)

print(f"Файл .npy сохранён: {anchor_embeddings_file}")
print(f"Файл антигерои в векторах сохранён: {antagonists_embeddings_file}")

triplets_extended_df = house_df[["anchor", "response", "neg_response"]]

# Проверяем, что в триплетах нет NaN перед сохранением
assert triplets_extended_df.isna().sum().sum() == 0, "Ошибка: В триплетах есть NaN!"

# Сохранение триплетов в pickle-формате
with open(triplets_path, "wb") as f:
    pickle.dump(triplets_extended_df, f)

print(f"Триплеты сохранены в: {triplets_path}")

# Автоматическая загрузка файлов для скачивания
files.download(triplets_path)